In [166]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler, OrdinalEncoder, LabelEncoder
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.metrics import matthews_corrcoef, mean_squared_error, mean_absolute_error, roc_curve, auc, roc_auc_score, recall_score, accuracy_score, classification_report, confusion_matrix, log_loss, ConfusionMatrixDisplay
from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.feature_selection import mutual_info_regression
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_breast_cancer, make_blobs
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBRegressor, XGBClassifier
import xgboost as xgb
import seaborn as sns
sns.set_theme(context='notebook')
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
# from imblearn.over_sampling import SMOTE

import itertools
import os 
os.chdir('e:/ml_data/kaggle_data/future_sale')

In [2]:
os.listdir()

['items.csv',
 'item_categories.csv',
 'sales_train.csv',
 'sample_submission.csv',
 'shops.csv',
 'test.csv']

In [25]:
train = pd.read_csv('sales_train.csv')
item_info = pd.read_csv('items.csv')

In [ ]:
train.info(show_counts=True)

In [24]:
train.nunique(axis=0)

date               1034
date_block_num       34
shop_id              60
item_id           21807
item_price        19993
item_cnt_day        198
dtype: int64

In [20]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.000,1.000
1,03.01.2013,0,25,2552,899.000,1.000
2,05.01.2013,0,25,2552,899.000,-1.000
3,06.01.2013,0,25,2554,1709.050,1.000
4,15.01.2013,0,25,2555,1099.000,1.000


In [39]:
item_info[['item_id', 'item_category_id']].head()

,item_id,item_category_id
0,0,40
1,1,76
2,2,40
3,3,40
4,4,40


In [41]:
train['item_id'].nunique()

21807

In [42]:
item_info['item_id'].nunique()

22170

In [43]:
train['item_category_id'] = train['item_id']

In [46]:
item_cate_id_dict = {item_id:category for item_id, category in zip(item_info['item_id'], item_info['item_category_id'])}

In [50]:
train['item_category_id'] = train['item_category_id'].map(item_cate_id_dict)

In [62]:
train.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 7 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   date              2935849 non-null  object 
 1   date_block_num    2935849 non-null  int64  
 2   shop_id           2935849 non-null  int64  
 3   item_id           2935849 non-null  int64  
 4   item_price        2935849 non-null  float64
 5   item_cnt_day      2935849 non-null  float64
 6   item_category_id  2935849 non-null  int64  
dtypes: float64(2), int64(4), object(1)
memory usage: 156.8+ MB


In [64]:
train.describe(exclude=['object'])

,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id
count,2935849.000,2935849.000,2935849.000,2935849.000,2935849.000,2935849.000
mean,14.570,33.002,10197.227,890.853,1.243,40.001
std,9.423,16.227,6324.297,1729.800,2.619,17.101
min,0.000,0.000,0.000,-1.000,-22.000,0.000
25%,7.000,22.000,4476.000,249.000,1.000,28.000
50%,14.000,31.000,9343.000,399.000,1.000,40.000
75%,23.000,47.000,15684.000,999.000,1.000,55.000
max,33.000,59.000,22169.000,307980.000,2169.000,83.000


In [90]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id
0,02.01.2013,0,59,22154,999.000,1.000,37
1,03.01.2013,0,25,2552,899.000,1.000,58
2,05.01.2013,0,25,2552,899.000,-1.000,58
3,06.01.2013,0,25,2554,1709.050,1.000,58
4,15.01.2013,0,25,2555,1099.000,1.000,56


In [54]:
test = pd.read_csv('test.csv')

In [91]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [127]:
df_item_month = pd.DataFrame(train.groupby(['date_block_num', 'shop_id', 'item_id'])['item_cnt_day'].sum())
df_item_month = df_item_month.rename({'item_cnt_day': 'item_cnt_month'}, axis=1)

In [135]:
df_item_month['item_cnt_month']

date_block_num  shop_id  item_id
0               0        32        6.000
                         33        3.000
                         35        1.000
                         43        1.000
                         51        2.000
                                    ... 
33              59       22087     6.000
                         22088     2.000
                         22091     1.000
                         22100     1.000
                         22102     1.000
Name: item_cnt_month, Length: 1609124, dtype: float64

In [136]:
df_item_price = pd.DataFrame(train.groupby(["date_block_num", "shop_id", "item_id"])["item_price"].min())

In [138]:
df_train = df_item_month.join(df_item_price, on=["date_block_num","shop_id","item_id"])

In [140]:
df_train = df_train.reset_index(level=[0,1,2])

In [141]:
df_train

,date_block_num,shop_id,item_id,item_cnt_month,item_price
0,0,0,32,6.000,221.000
1,0,0,33,3.000,347.000
2,0,0,35,1.000,247.000
3,0,0,43,1.000,221.000
4,0,0,51,2.000,127.000
...,...,...,...,...,...
1609119,33,59,22087,6.000,119.000
1609120,33,59,22088,2.000,119.000
1609121,33,59,22091,1.000,179.000
1609122,33,59,22100,1.000,629.000


In [144]:
df_train = df_train.join(item_info, on="item_id", rsuffix="_ITEMS")

In [ ]:
del(df_train["item_id_ITEMS"])

In [148]:
df_train

,date_block_num,shop_id,item_id,item_cnt_month,item_price,item_name,item_category_id
0,0,0,32,6.000,221.000,1+1,40
1,0,0,33,3.000,347.000,1+1 (BD),37
2,0,0,35,1.000,247.000,10 ЛЕТ СПУСТЯ,40
3,0,0,43,1.000,221.000,100 МИЛЛИОНОВ ЕВРО,40
4,0,0,51,2.000,127.000,100 лучших произведений классики (mp3-CD) (Dig...,57
...,...,...,...,...,...,...,...
1609119,33,59,22087,6.000,119.000,Элемент питания DURACELL LR03-BC2,83
1609120,33,59,22088,2.000,119.000,Элемент питания DURACELL LR06-BC2,83
1609121,33,59,22091,1.000,179.000,Элемент питания DURACELL TURBO LR 03 2*BL,83
1609122,33,59,22100,1.000,629.000,Энциклопедия Adventure Time,42


In [149]:
df_test = test.join(item_info, on="item_id", rsuffix="_ITEMS")
df_test = df_test.drop(["item_id_ITEMS", "item_id_ITEMS"], axis=1)
df_test.head()


,ID,shop_id,item_id,item_name,item_category_id
0,0,5,5037,"NHL 15 [PS3, русские субтитры]",19
1,1,5,5320,ONE DIRECTION Made In The A.M.,55
2,2,5,5233,"Need for Speed Rivals (Essentials) [PS3, русск...",19
3,3,5,5232,"Need for Speed Rivals (Classics) [Xbox 360, ру...",23
4,4,5,5268,"Need for Speed [PS4, русская версия]",20


In [153]:
df_train.head()

,date_block_num,shop_id,item_id,item_cnt_month,item_price,item_name,item_category_id
0,0,0,32,6.000,221.000,1+1,40
1,0,0,33,3.000,347.000,1+1 (BD),37
2,0,0,35,1.000,247.000,10 ЛЕТ СПУСТЯ,40
3,0,0,43,1.000,221.000,100 МИЛЛИОНОВ ЕВРО,40
4,0,0,51,2.000,127.000,100 лучших произведений классики (mp3-CD) (Dig...,57


In [154]:
from scipy import stats

df_train["item_cnt_month"] = df_train["item_cnt_month"].map(lambda x: abs(x))
df_train = df_train[(np.abs(stats.zscore(df_train["item_cnt_month"])) < 3)]


In [156]:
df = pd.DataFrame(df_train.groupby("item_cnt_month")["item_price"].agg(['mean','count']))
df = df.astype('int32')

In [159]:
def multiply(x,y):
    return x*y

df_train['total_price'] = multiply(df_train["item_price"], df_train["item_cnt_month"])

C:\Users\fxinw\AppData\Local\Temp\ipykernel_99124\2766275814.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['total_price'] = multiply(df_train["item_price"], df_train["item_cnt_month"])


In [160]:
df = pd.DataFrame(df_train.groupby("item_cnt_month")["total_price"].agg(['sum']))
df = df.astype('int32')


In [162]:
features = ["item_id", "shop_id"]
X = df_train[features]
y = df_train["item_cnt_month"]


In [164]:
df_train[['item_id', 'shop_id', 'item_cnt_month']]

,item_id,shop_id,item_cnt_month
0,32,0,6.000
1,33,0,3.000
2,35,0,1.000
3,43,0,1.000
4,51,0,2.000
...,...,...,...
1609119,22087,59,6.000
1609120,22088,59,2.000
1609121,22091,59,1.000
1609122,22100,59,1.000


In [165]:
x_train, x_valid, y_train, y_valid = train_test_split(X, y, random_state=1, test_size=0.2)

In [170]:
model = LogisticRegression(random_state=1)

model.fit(x_train, y_train)

predictions = model.predict(x_valid)
predictions = predictions.round()

msq = mean_squared_error(predictions, y_valid)


D:\miniconda\envs\ptc\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [181]:
test[['shop_id', 'item_id']].nunique()

shop_id      42
item_id    5100
dtype: int64

In [186]:
df_train['shop_id'].nunique()

60